In [ ]:
%pip install sentence-transformers

In [14]:
%pip install replicate
%pip install pypdf
%pip install langchain==0.2.0
%pip install langchain_pinecone==0.0.7
%pip install -U langchain-community
%pip install pinecone-client


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached langchain-0.2.0-py3-none-any.whl.metadata (13 kB)
Using cached langchain-0.2.0-py3-none-any.whl (973 kB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.7
    Uninstalling langchain-0.2.7:
      Successfully uninstalled langchain-0.2.7
Note: you may need to restart the kernel to use updated packages.


...

ERROR: Could not find a version that satisfies the requirement langchain_pinecone==0.0.7 (from versions: 0.0.1, 0.0.2rc0, 0.0.2, 0.0.3, 0.1.0, 0.1.1)
ERROR: No matching distribution found for langchain_pinecone==0.0.7

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached langchain-0.2.7-py3-none-any.whl.metadata (6.9 kB)
Using cached langchain-0.2.7-py3-none-any.whl (983 kB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.0
    Uninstalling langchain-0.2.0:
      Successfully uninstalled langchain-0.2.0
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import pinecone
import replicate
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community .llms import CTransformers
from langchain.vectorstores import Pinecone

In [17]:
PINECONE_API_KEY = "c85a2244-4db3-464b-a9e2-f366e296718d"
PINECONE_API_ENV = "us-east-1"

In [35]:
def load_pdf(data):    
        loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader)
        
        documents = loader.load()
        return documents
        print(documents)
        

In [36]:
extracted_data = load_pdf("./data/")

In [ ]:
extracted_data

In [38]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [39]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 1911


In [40]:
text_chunks

[Document(metadata={'source': 'data\\Vegetable Gardeners Bible - Edward.C.Smith.pdf', 'page': 1}, page_content='T h e  V e g e t a b l e  G a r d e n e r ’ s  B I B L E\nSECOND EDITION\nDiscover Ed’ s High-Y ield W -O-R-D System for All North\nAmerican Gardening Regions\nEdward C. Smith'),
 Document(metadata={'source': 'data\\Vegetable Gardeners Bible - Edward.C.Smith.pdf', 'page': 2}, page_content='DEDICA TION\nT o my darling Sylvia,\nwith whom I have gardened joyously\nfor more than thirty years.\nThe mission of Storey Publishing is to serve our customers by publishing\npractical information that encourages personal independence in harmony\nwith the environment.\nEdited by Carleen Madigan and Gwen Steege\nArt direction and book design by Jessica Armstrong\nFront cover photography by © Storey Publishing/Giles Prett (author)\nand © W alter Chandoha (background)'),
 Document(metadata={'source': 'data\\Vegetable Gardeners Bible - Edward.C.Smith.pdf', 'page': 2}, page_content='Back cover 

In [24]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [25]:
embeddings = download_hugging_face_embeddings()

c:\Users\Admin\Desktop\Graduating Project\GenAI\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [30]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
%pip install langchain_pinecone

In [41]:
#Initializing the Pinecone
import os
from pinecone import Pinecone

os.environ['PINECONE_API_KEY']= 'c85a2244-4db3-464b-a9e2-f366e296718d'

pinecone.instance = Pinecone(api_key='c85a2244-4db3-464b-a9e2-f366e296718d',environment='us-east-1')

index_name="graduating-project"

from langchain_pinecone import Pinecone

#Creating Embeddings for Each of The Text Chunks & storing
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [42]:
index_name="graduating-project"
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "How to manage a farm"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='One fina l important bit of advice : Remember to pass on what you know to a\nfuture sm all farm er . T eaching others is also a great way of reinf orcing what\nyou kno w . By being honest about the joys and pitfalls of farmi ng, you can\nmentor others about the wonderful world of small-scale farming.\nT able of Contents'), Document(page_content='One fina l important bit of advice : Remember to pass on what you know to a\nfuture sm all farm er . T eaching others is also a great way of reinf orcing what\nyou kno w . By being honest about the joys and pitfalls of farmi ng, you can\nmentor others about the wonderful world of small-scale farming.\nT able of Contents'), Document(page_content='clearly\tmakes\tsense\tand\tis\tworth\trepeating—go\tand\twork\ton\ta\tfarm.\tTry\tout\nthe\tidea\tby\tlaboring\t(and\tlearning)\twith\tsomeone\telse.\tExperience\tthe\tgood\ntimes\tand\tthe\tbad,\tthe\trealities\tand\tthe\trewards.\tIf\tpossible,\twork\ton\tmore\nthan\ton

In [44]:
def load_pinecone_existing_index():
    docsearch = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)
    return docsearch
docsearch=load_pinecone_existing_index()

query = "How to manage a farm"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='One fina l important bit of advice : Remember to pass on what you know to a\nfuture sm all farm er . T eaching others is also a great way of reinf orcing what\nyou kno w . By being honest about the joys and pitfalls of farmi ng, you can\nmentor others about the wonderful world of small-scale farming.\nT able of Contents'), Document(page_content='One fina l important bit of advice : Remember to pass on what you know to a\nfuture sm all farm er . T eaching others is also a great way of reinf orcing what\nyou kno w . By being honest about the joys and pitfalls of farmi ng, you can\nmentor others about the wonderful world of small-scale farming.\nT able of Contents'), Document(page_content='clearly\tmakes\tsense\tand\tis\tworth\trepeating—go\tand\twork\ton\ta\tfarm.\tTry\tout\nthe\tidea\tby\tlaboring\t(and\tlearning)\twith\tsomeone\telse.\tExperience\tthe\tgood\ntimes\tand\tthe\tbad,\tthe\trealities\tand\tthe\trewards.\tIf\tpossible,\twork\ton\tmore\nthan\ton

In [45]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [46]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [47]:
from langchain.llms import Replicate
os.environ['REPLICATE_API_TOKEN']="r8_YgNQpNUeSV2XL230IieDlrWC1YU0KFR3eQjJz"
llm = Replicate(model= "llama-2-7b-chat.ggmlv3.q4_0.bin")

In [48]:
retriever=docsearch.as_retriever(search_kwargs={'k': 2})
print(retriever)

tags=['Pinecone', 'HuggingFaceEmbeddings'] vectorstore=<langchain_pinecone.vectorstores.Pinecone object at 0x000001D3108B38C0> search_kwargs={'k': 2}


In [49]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

: 